In [ ]:
# ============================================================
# PYSPARK PLAYER BEHAVIOR CLUSTERING & ANALYSIS
# ============================================================

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, mean
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans

In [ ]:
# ============================================================
# 1️⃣ KHỞI TẠO SPARK
# ============================================================

spark = SparkSession.builder \
    .appName("PlayerBehaviorClustering") \
    .getOrCreate()

In [ ]:
# ============================================================
# 2️⃣ LOAD DỮ LIỆU
# ============================================================

df = spark.read.csv("data/online_gaming_behavior_dataset.csv", header=True, inferSchema=True)
print("✅ Dữ liệu gốc:")
df.show(5)
df.printSchema()

In [ ]:
# ============================================================
# 3️⃣ KHÁM PHÁ DỮ LIỆU (EDA)
# ============================================================

print("🔹 Thống kê tổng quan:")
df.describe().show()

print("🔹 Phân bố giới tính:")
df.groupBy("Gender").count().show()

print("🔹 Phân bố theo khu vực:")
df.groupBy("Location").count().show()

print("🔹 Phân bố thể loại game:")
df.groupBy("GameGenre").count().show()


In [ ]:
# ============================================================
# 4️⃣ TIỀN XỬ LÝ DỮ LIỆU
# ============================================================

categorical_cols = ["Gender","Location","GameGenre","GameDifficulty"]
indexers = [StringIndexer(inputCol=col, outputCol=col+"_idx") for col in categorical_cols]
pipeline = Pipeline(stages=indexers)
df_indexed = pipeline.fit(df).transform(df)

In [ ]:
# ============================================================
# 5️⃣ CHUẨN BỊ DỮ LIỆU CHO CLUSTERING
# ============================================================

feature_cols = ["Age", "PlayTimeHours", "SessionsPerWeek", "AvgSessionDurationMinutes",
                "PlayerLevel", "AchievementsUnlocked"] + [col+"_idx" for col in categorical_cols]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df_features = assembler.transform(df_indexed)


In [ ]:
# ============================================================
# 6️⃣ PHÂN CỤM (K-MEANS)
# ============================================================

kmeans = KMeans(k=3, seed=42, featuresCol="features", predictionCol="cluster")
model = kmeans.fit(df_features)
df_clustered = model.transform(df_features)

print("✅ Gán cụm cho người chơi:")
df_clustered.select("PlayerID","cluster").show(10)

In [ ]:
# ============================================================
# 7️⃣ PHÂN TÍCH CỤM
# ============================================================

print("🔹 Thống kê trung bình các chỉ số theo cụm:")
df_clustered.groupBy("cluster").mean("Age","PlayTimeHours","SessionsPerWeek",
                                     "AvgSessionDurationMinutes","PlayerLevel","AchievementsUnlocked").show()

print("🔹 Phân bố categorical theo cụm:")
for col_name in ["Gender","Location","GameGenre","GameDifficulty"]:
    print(f"\n📊 Phân bố {col_name} theo cụm:")
    df_clustered.groupBy("cluster", col_name).count().orderBy("cluster").show()


In [ ]:
# ============================================================
# 8️⃣ XUẤT KẾT QUẢ CHO REPORT
# ============================================================

# Xuất ra CSV cho slide/report
df_clustered.select("PlayerID","Age","Gender","Location","GameGenre","PlayTimeHours",
                    "SessionsPerWeek","AvgSessionDurationMinutes","PlayerLevel",
                    "AchievementsUnlocked","cluster") \
            .coalesce(1).write.csv("clustered_output", header=True, mode="overwrite")

print("✅ Kết quả đã được xuất ra thư mục: clustered_output/")


In [ ]:
# ============================================================
# 9️⃣ KẾT THÚC
# ============================================================

spark.stop()
